In [6]:
import numpy as np
from laspy.file import File
import pandas
import geopandas
from shapely.geometry import Point
from IPython.display import display, clear_output
import pptk
import osmium as osm

In [9]:
shape_file = "data/colonge_small/shp_koeln/gis_osm_buildings.shp"
shape  = geopandas.read_file(shape_file)

lidar_file = "data/colonge_small/dom1l-fp_32356_5645_1_nw.xyz"
#read the file into a numpy array
lidar_points = np.loadtxt(open(lidar_file, "r"), delimiter=",", skiprows=0)

In [39]:
max_index = shape.index.max()

for i in range(0, (max_index + 1)):
    current_object = shape.loc[i]
    
    shape_name = current_object['name']
    shape_type = current_object['type']
    
    if (shape_name == None and (shape_type == None or shape_type == "appartment" or shape_type == "house")):
        print(current_object['geometry'])

#print(shape)
#print(shape.keys())

#shape.loc[0]
#print(shape["osm_id"][0])

POLYGON ((356617.3888255692 5645534.691977032, 356620.1779379734 5645536.406295503, 356622.4720819072 5645532.681917631, 356619.6829684071 5645530.967597912, 356617.3888255692 5645534.691977032))
POLYGON ((356635.7616328746 5645231.442267929, 356636.9076244959 5645231.443944038, 356636.9207961125 5645233.190570186, 356640.9476358726 5645233.134799176, 356640.9552053063 5645237.474247714, 356639.1488694358 5645237.490839992, 356639.3102049144 5645245.609326154, 356642.858416063 5645245.533416109, 356642.8829174599 5645247.435511372, 356651.8733522188 5645247.387084951, 356651.9299389952 5645250.957391599, 356653.5386490782 5645250.912888615, 356655.5199882976 5645250.869205316, 356655.4665957512 5645252.751200589, 356668.0611181895 5645252.614229575, 356668.0889792121 5645256.163075848, 356677.0558620046 5645256.026325232, 356677.0443038642 5645257.896035518, 356695.1283531712 5645257.718570208, 356695.1747349658 5645255.83676987, 356704.115977008 5645255.789793754, 356704.0524762828 56

POLYGON ((356810.987785976 5646104.231256918, 356819.1136777761 5646103.984402551, 356818.9852157243 5646097.556357622, 356810.8459258133 5646098.081765276, 356810.987785976 5646104.231256918))
POLYGON ((356763.4049136646 5646101.340571936, 356763.7489112177 5646108.697355137, 356784.6726270116 5646107.607000535, 356784.1282584949 5646099.866300157, 356763.4049136646 5646101.340571936))
POLYGON ((356792.0692044008 5646086.572158167, 356792.4466054431 5646093.104595399, 356792.8126333812 5646099.225635868, 356803.5760491568 5646098.527516584, 356803.1962484932 5646092.162055407, 356802.8067027621 5646085.952646272, 356792.0692044008 5646086.572158167))
POLYGON ((356791.8241734088 5646063.723395972, 356794.9510894034 5646083.009637045, 356799.435918653 5646082.763261092, 356796.5276455735 5646065.028800191, 356795.4072939565 5646063.913655519, 356796.1211291814 5646063.037118343, 356791.8241734088 5646063.723395972))
POLYGON ((356779.8101181953 5646094.700198456, 356780.1476193852 564610

POLYGON ((356092.3802538736 5645385.952818692, 356094.2267050341 5645396.49480783, 356099.8372424727 5645396.005168941, 356099.9470874077 5645390.594209548, 356099.9595156305 5645390.282297633, 356099.7551745132 5645381.152390054, 356099.6731144626 5645378.450714648, 356095.2979919728 5645378.87266003, 356095.4935335381 5645383.129017659, 356092.8654210548 5645383.42455514, 356092.4284231116 5645385.662168875, 356092.3802538736 5645385.952818692))
POLYGON ((356240.0576857206 5645059.61604342, 356243.186060864 5645059.607149522, 356243.0848729661 5645063.816100059, 356250.2048608473 5645063.996918746, 356250.2045504536 5645061.704690576, 356250.2220552533 5645052.95809776, 356240.664467414 5645050.063059545, 356240.4977879593 5645052.671486597, 356240.0576857206 5645059.61604342))
POLYGON ((356570.3207218418 5645732.158640658, 356570.8245063218 5645734.35903558, 356572.0252952961 5645734.058736974, 356572.2652655931 5645735.109191486, 356572.6589330994 5645735.109420717, 356572.59627488

POLYGON ((355969.4551012044 5646082.271739209, 355977.0392875165 5646086.556355472, 355982.5085559065 5646089.608998174, 355988.47746112 5646079.027807883, 355975.3965129372 5646071.713552394, 355969.4551012044 5646082.271739209))
POLYGON ((356013.0020197701 5646111.127441552, 356021.9382592187 5646116.008802266, 356029.611051194 5646101.986446457, 356020.6930865648 5646097.004417302, 356013.07557285 5646110.99186827, 356013.0020197701 5646111.127441552))
POLYGON ((355855.2419400447 5646043.809653997, 355864.917160347 5646049.471368243, 355867.3468884744 5646045.342258371, 355869.9636996593 5646040.862994833, 355869.5131520118 5646040.586216987, 355860.2594462853 5646035.168696324, 355855.2419400447 5646043.809653997))
POLYGON ((356037.4736272193 5646125.903280476, 356043.9104079058 5646129.374193418, 356046.8931517159 5646123.894515626, 356040.4078182497 5646120.447202873, 356037.4736272193 5646125.903280476))
POLYGON ((356238.6675934708 5645211.231581221, 356239.1913200135 5645218.70

POLYGON ((356277.7603344804 5646099.60252991, 356279.078078433 5646100.734346043, 356278.3846923106 5646102.34479846, 356287.469125985 5646106.243284748, 356287.6876472504 5646105.758745225, 356289.5158938483 5646106.542580215, 356291.9612018702 5646102.457760352, 356287.6124610485 5646100.007975521, 356281.1587855268 5646095.669299279, 356280.3549634308 5646096.592917578, 356279.9951363577 5646096.291333658, 356278.4558801678 5646098.069909179, 356278.808991899 5646098.382806679, 356277.7603344804 5646099.60252991))
POLYGON ((355844.3625767266 5645705.861091922, 355847.0387234439 5645723.468095692, 355848.3619877656 5645723.275494308, 355848.9050613977 5645726.87679443, 355861.659428186 5645724.941851462, 355862.1905188336 5645728.365447293, 355865.9700344581 5645727.770694309, 355863.2840163199 5645710.564545227, 355863.2105895411 5645710.199383309, 355852.6837354687 5645711.849797583, 355852.0164282507 5645707.573181356, 355851.5566826876 5645704.692844635, 355844.3625767266 5645705

POLYGON ((356409.9488353403 5645979.811627025, 356413.9117012754 5645986.344826553, 356416.1089514344 5645984.970898136, 356415.6932943972 5645983.669389389, 356416.00444132 5645983.482727809, 356416.1109053525 5645983.012427726, 356412.9628952204 5645977.98109062, 356409.9488353403 5645979.811627025))
POLYGON ((355860.0862673857 5645172.309461802, 355860.6776983633 5645181.439736387, 355860.7988916457 5645183.016456611, 355864.1821806616 5645182.82220171, 355864.3820609326 5645186.72236059, 355864.4333565418 5645187.555489048, 355868.4089108669 5645187.422654601, 355868.0303745302 5645172.043985115, 355860.0862673857 5645172.309461802))
POLYGON ((356166.7465985994 5645686.842036442, 356167.6897811356 5645688.151138488, 356169.1905149957 5645689.033049314, 356170.6445560257 5645688.992683958, 356172.0443382101 5645688.263927503, 356172.6477496979 5645686.956400157, 356172.5630083072 5645685.423173735, 356173.8202187431 5645684.876413776, 356174.7217865215 5645686.96558906, 356182.08172